In [4]:
# wymagane biblioteki
import pandas as pd
import re

In [ ]:
# wczytywanie danych
df_org = pd.read_excel('scraping results/excel/dane_otomoto.xlsx')
df_org.shape

In [ ]:
# funkcja do oczyszczania danych
def get_data(df_org):
    df = df_org[[' Pierwszy właściciel (od nowości)', 'Bezwypadkowy', 'Cena', 'Kierownica po prawej (Anglik)', 'Kolor', 'Kraj pochodzenia', 'Liczba drzwi', 'Liczba miejsc', 'Marka pojazdu', 'Moc', 'Model pojazdu', 'Napęd', 'Oferta od', 'Pierwszy właściciel', 'Pojemność skokowa', 'Przebieg', 'Rodzaj paliwa', 'Rok produkcji', 'Skrzynia biegów', 'Serwisowany w ASO', 'Spalanie W Mieście', 'Stan', 'Tuning', 'Typ nadwozia', 'Uszkodzony', 'Zarejestrowany jako zabytek', 'Zarejestrowany w Polsce']]
    df = df.dropna(subset=['Cena', 'Moc', 'Pojemność skokowa', 'Przebieg', 'Spalanie W Mieście'])
    df = df.dropna(subset=['Liczba drzwi', 'Liczba miejsc', 'Napęd', 'Skrzynia biegów'])

    df = df[df['Rok produkcji'] > 1970].copy()

    # tworzenie i przetwarzanie zmiennych
    new_cena = df['Cena'].apply( lambda x: int(re.search(r'[\d ]+', x).group().replace(' ', '')))
    df.insert(1, 'cena', new_cena)
    waluta = df['Cena'].apply( lambda x: re.search(r'[A-Z]{3}', x).group().replace(' ', ''))#.map({'PLN': 0, 'EUR': 1})
    df.insert(2,'waluta', waluta)
    moc = df['Moc'].apply( lambda x: int(re.search(r'[\d ]+', x).group().replace(' ', '')))
    df.insert(3,'moc', moc)
    pojemnosc = df['Pojemność skokowa'].apply( lambda x: int(re.search(r'[\d ]+', x).group().replace(' ', '')))
    df.insert(4,'pojemnosc_cm3', pojemnosc)
    przebieg = df['Przebieg'].apply( lambda x: int(re.search(r'[\d ]+', x).group().replace(' ', '')))
    df.insert(5,'przebieg_km', przebieg)
    spalanie = df['Spalanie W Mieście'].apply( lambda x: float(re.search(r'[\d,]+', x).group().replace(',', '.')))
    df.insert(6,'spalanie_(l/100km)', spalanie)
    df.insert(7, 'wiek', 2022-df['Rok produkcji'])

    df['Pierwszy właściciel'] = df[' Pierwszy właściciel (od nowości)']
    df = df.drop(columns=['Cena', 'Moc', 'Pojemność skokowa', 'Spalanie W Mieście', 'Przebieg', ' Pierwszy właściciel (od nowości)'])

    df['Kraj pochodzenia'].fillna('Polska', inplace = True)
    df = df[(df['Marka pojazdu'] != 'Porsche') & (df['Marka pojazdu'] != 'Land Rover')].copy()

    # removing outliers
    df = df[df['cena'] < 500000].copy()
    df = df[df['przebieg_km'] < 6E5].copy()
    df = df[df['pojemnosc_cm3'] < 6500].copy()
    df = df[df['spalanie_(l/100km)'] < 25].copy()
    df = df[df['Rok produkcji'] > 1960].copy()
    df = df[df['moc'] < 500].copy()

    df = df[df['Stan'] != 'Nowe']

    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].str.strip().str.lower().str.replace(' ', '_')

    # poprawa nazw kolumn
    column_names = [
        'cena',
        'waluta',
        'moc_km',
        'pojemnosc_cm3',
        'przebieg_km',
        'spalanie',
        'wiek',
        'bezwypadkowy',
        'kierownica_po_prawej',
        'kolor',
        'kraj_pochodzenia',
        'liczba_drzwi',
        'liczba_miejsc',
        'marka',
        'model',
        'naped',
        'oferta_od',
        'pierwszy_wlasciciel',
        'paliwo',
        'rok',
        'skrzynia',
        'serwisowany_w_aso',
        'stan',
        'tuning',
        'nadwozie',
        'uszkodzony',
        'zabytek',
        'zarejestrowany_w_pl'
    ]
    df.columns = column_names
    df['naped'] = df['naped'].replace(['4x4 (stały)', '4x4 (dołączany automatycznie)', '4x4 (dołączany ręcznie)'], '4x4')

    return df.reset_index(drop=True)
df = get_data(df_org)

df.shape

In [ ]:
df.to_excel('cars_clean.xlsx')
df.to_parquet('cars_clean.parquet')